<a href="https://colab.research.google.com/github/bashirnubtk/Virtual-CPU-Emulator/blob/main/week%207/W7_Advanced_CPU_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Week 7: Advanced Features
# Integrated with Week 6 functionality for I/O and prior weeks' memory management and instruction execution

# Instruction Set Definitions
INSTRUCTION_SET = {
    'LOAD': '0001',
    'STORE': '0010',
    'ADD': '0011',
    'SUB': '0100',
    'JMP': '0101',
    'HALT': '1111',
    'BRANCH': '0110',  # New instruction for branching
    'CALL': '0111',    # New instruction for subroutines
    'RET': '1000'      # Return from subroutine
}

# Register Definitions
REGISTER_SET = {'R0': 0, 'R1': 1, 'R2': 2, 'R3': 3}

# Memory Class for Simulated Memory Management
class Memory:
    def __init__(self, size):
        self.memory = [0] * size

    def read(self, address):
        if 0 <= address < len(self.memory):
            return self.memory[address]
        else:
            raise IndexError(f"Invalid memory address: {address}")

    def write(self, address, value):
        if 0 <= address < len(self.memory):
            self.memory[address] = value
        else:
            raise IndexError(f"Invalid memory address: {address}")

    def display_memory(self):
        for i in range(0, len(self.memory), 8):
            chunk = self.memory[i:i+8]
            print(f"{i:04}: " + " ".join(f"{v:08b}" for v in chunk))

# CPU Class with Advanced Features
class CPUWithAdvancedFeatures:
    def __init__(self, memory_size):
        self.memory = Memory(memory_size)
        self.registers = [0] * 4
        self.program_counter = 0
        self.stack = []
        self.halted = False

    def load_program(self, program):
        for i, instruction in enumerate(program):
            self.memory.write(i, instruction)

    def fetch_instruction(self):
        if self.program_counter < len(self.memory.memory):
            instruction = self.memory.read(self.program_counter)
            self.program_counter += 1
            return f"{instruction:016b}"
        else:
            raise IndexError(f"Program counter out of memory bounds: {self.program_counter}")

    def decode_execute(self, instruction):
        opcode = instruction[:4]
        operands = instruction[4:]

        if opcode == INSTRUCTION_SET['LOAD']:
            reg = int(operands[:2], 2)
            addr = int(operands[2:], 2)
            self.registers[reg] = self.memory.read(addr)

        elif opcode == INSTRUCTION_SET['STORE']:
            reg = int(operands[:2], 2)
            addr = int(operands[2:], 2)
            self.memory.write(addr, self.registers[reg])

        elif opcode == INSTRUCTION_SET['ADD']:
            reg1 = int(operands[:2], 2)
            reg2 = int(operands[2:], 2)
            self.registers[reg1] += self.registers[reg2]

        elif opcode == INSTRUCTION_SET['SUB']:
            reg1 = int(operands[:2], 2)
            reg2 = int(operands[2:], 2)
            self.registers[reg1] -= self.registers[reg2]

        elif opcode == INSTRUCTION_SET['JMP']:
            addr = int(operands, 2)
            if 0 <= addr < len(self.memory.memory):
                self.program_counter = addr
            else:
                raise IndexError(f"Invalid JMP address: {addr}")

        elif opcode == INSTRUCTION_SET['HALT']:
            self.halted = True

        elif opcode == INSTRUCTION_SET['BRANCH']:
            reg = int(operands[:2], 2)
            addr = int(operands[2:], 2)
            if self.registers[reg] != 0 and 0 <= addr < len(self.memory.memory):
                self.program_counter = addr

        elif opcode == INSTRUCTION_SET['CALL']:
            addr = int(operands, 2)
            if 0 <= addr < len(self.memory.memory):
                self.stack.append(self.program_counter)
                self.program_counter = addr
            else:
                raise IndexError(f"Invalid CALL address: {addr}")

        elif opcode == INSTRUCTION_SET['RET']:
            if self.stack:
                self.program_counter = self.stack.pop()
            else:
                raise IndexError("Stack underflow during RET")

    def run(self):
        while not self.halted:
            try:
                instruction = self.fetch_instruction()
                self.decode_execute(instruction)
            except IndexError as e:
                print(e)
                self.halted = True

# Example Program for Testing
program_binary = [
    int('0001000100000010', 2),  # LOAD R1, 2
    int('0001001000000110', 2),  # LOAD R2, 6
    int('0011000100100000', 2),  # ADD R1, R2
    int('0010000100011110', 2),  # STORE R1, 14
    int('0111000000001110', 2),  # CALL 14
    int('1000000000000000', 2),  # RET
    int('1111000000000000', 2),  # HALT
]

# Initialize CPU and Memory
cpu = CPUWithAdvancedFeatures(memory_size=32)  # Increased memory size to 32 bytes
cpu.load_program(program_binary)
cpu.run()

# Display Memory Contents
cpu.memory.display_memory()


Invalid memory address: 258
0000: 1000100000010 1001000000110 11000100100000 10000100011110 111000000001110 1000000000000000 1111000000000000 00000000
0008: 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000
0016: 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000
0024: 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000
